In [ ]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/feedback-prize-2021/train.csv')
train.shape

(144293, 8)

In [ ]:
full_essays_data= pd.read_csv('/content/drive/MyDrive/feedback-prize-2021/train_text_df.csv')
full_essays_data.head()

,id,text
0,F00B4D036D97,"First and formost, it is my belief that we sho..."
1,EFDCE6A9E77B,The principal has made a decision saying all s...
2,EFE2D4E09A10,"Dear, Senator\n\nThe\n\nElectoral college has ..."
3,EFE800DEFA4B,I think that the use of this technology to rea...
4,EFDEF0E460D1,Not A Face\n\nThe Face on Mars is just a natur...


In [ ]:
train.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [ ]:
# Take 20% of the rows using the 'sample' method
df = full_essays_data.sample(frac=0.2)
# Display the DataFrame
df.head(20)

,id,text
7936,8CAF1B7BD82D,As technology advances in a very constant and ...
834,FD361491E6FA,While i feel like this could help with a few t...
4602,B71521CCA4F4,I think that this new rule is necessary.\n\nHa...
14412,10B28752BA07,"""The Challenge of Exploring Venus"" talks about..."
2000,CF52358A03FF,Of all things that you would want to learn abo...
7254,814488D5C66D,Using cars today is polluting the air we breat...
6415,94381B3CEBFA,"Dear TEACHER_NAME,\n\nI'm writing this letter ..."
1731,EB49C042DB20,"Dear Principal,\n\nWhat I think you have chose..."
630,FA363FC0B71E,won't it be so cool to have a car that drives ...
1918,EEADACB875D7,Phones and Driving\n\nDistractions can serve a...


In [ ]:
#creating the test data for last 20% of the essays
import pandas as pd
import nltk

# Download the Punkt tokenizer model (used for sentence tokenization)
nltk.download('punkt')

# Function to tokenize the essay text into sentences
def tokenize_sentences(text):
    return nltk.sent_tokenize(text)

# Function to extract word index numbers in the entire essay text
def get_word_indices(text):
    words = nltk.word_tokenize(text)
    word_indices = []
    count = 0
    for word in words:
        word_indices.append(count)
        count += 1
    return word_indices


# Create a new DataFrame to store the results
processed_test_df = pd.DataFrame(columns=['discourse_text', 'predictionstring'])

# Iterate through each essay in the original DataFrame
for _, row in df.iterrows():
    essay_id = row['id']
    sentences = tokenize_sentences(row['text'])
    start_index = 0

    for sentence in sentences:
        word_indices = get_word_indices(sentence)
        end_index = start_index + len(word_indices)
        processed_test_df = processed_test_df.append({'id': essay_id, 'discourse_text': sentence, 'predictionstring': list(range(start_index, end_index))}, ignore_index=True)
        start_index = end_index



Streaming output truncated to the last 5000 lines.
<ipython-input-8-78048ea057e1>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_test_df = processed_test_df.append({'id': essay_id, 'discourse_text': sentence, 'predictionstring': list(range(start_index, end_index))}, ignore_index=True)
<ipython-input-8-78048ea057e1>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_test_df = processed_test_df.append({'id': essay_id, 'discourse_text': sentence, 'predictionstring': list(range(start_index, end_index))}, ignore_index=True)
<ipython-input-8-78048ea057e1>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processed_test_df = processed_test_df.append({'id': essay_id, 'discourse_text': sentence, 'predictionstri

In [ ]:
processed_test_df['predictionstring'] = processed_test_df['predictionstring'].astype(object)
# Display the updated DataFrame
processed_test_df.head(20)

,discourse_text,predictionstring,id
0,As technology advances in a very constant and ...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",8CAF1B7BD82D
1,While we still might be waiting for the next f...,"[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3...",8CAF1B7BD82D
2,An example of this is a computer software that...,"[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 5...",8CAF1B7BD82D
3,Profesor Thomas Huang from the Beckman Institu...,"[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 7...",8CAF1B7BD82D
4,"Among the several other practical uses, Huang ...","[97, 98, 99, 100, 101, 102, 103, 104, 105, 106...",8CAF1B7BD82D
5,Overall the use of emotional recognition has t...,"[121, 122, 123, 124, 125, 126, 127, 128, 129, ...",8CAF1B7BD82D
6,"Dr. Huang, the inventor of the emotional recog...","[172, 173, 174, 175, 176, 177, 178, 179, 180, ...",8CAF1B7BD82D
7,"In a time of computer screens, video games, an...","[198, 199, 200, 201, 202, 203, 204, 205, 206, ...",8CAF1B7BD82D
8,Dr. Huang's main goal is to put a computer wit...,"[223, 224, 225, 226, 227, 228, 229, 230, 231, ...",8CAF1B7BD82D
9,If a student begins to get bored and their zyg...,"[255, 256, 257, 258, 259, 260, 261, 262, 263, ...",8CAF1B7BD82D


In [ ]:
processed_test_df['predictionstring'] = processed_test_df['predictionstring'].apply(lambda x: ' '.join([str(num) for num in x]))
# Display the DataFrame
processed_test_df.head()

,discourse_text,predictionstring,id
0,As technology advances in a very constant and ...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18,8CAF1B7BD82D
1,While we still might be waiting for the next f...,19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 3...,8CAF1B7BD82D
2,An example of this is a computer software that...,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 6...,8CAF1B7BD82D
3,Profesor Thomas Huang from the Beckman Institu...,62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 7...,8CAF1B7BD82D
4,"Among the several other practical uses, Huang ...",97 98 99 100 101 102 103 104 105 106 107 108 1...,8CAF1B7BD82D


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
test_ids =list(processed_test_df['id'].unique())

# Create a new train DataFrame by removing the rows with IDs present in test_ids
new_train_df = train[~train['id'].isin(test_ids)]
new_train_df.reset_index(drop=True, inplace=True)
new_train_df.head()


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


## Preprocessing

In [ ]:
def clean_text(text, remove_stopwords=True, stem_words=False, lemma=True):
    text = str(text).lower().split()

    if remove_stopwords:
        stops = stopwords.words("english")
        text = [x for x in text if not x in stops]

    text = ' '.join(text)

    text = re.sub(r"[-()\"#/<>!@&;*:<>{}`'+=~%|.!?,_]", " ", text)
    text = re.sub(r"\]", " ", text)
    text = re.sub(r"\[", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"  ", " ", text)
    text = re.sub(r"   ", " ", text)
    text = re.sub(r"   ", " ", text)
    text = re.sub(r"0x00", "", text)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stem_words = [stemmer.stem(x) for x in text]
        text = " ".join(text)

    if lemma:
        text = text.split()
        lem = WordNetLemmatizer()
        lemmatized = [lem.lemmatize(x, "v") for x in text]
        text = " ".join(text)

    return text

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
%%time
new_train_df['cleaned_text'] = new_train_df.discourse_text.apply(clean_text)
processed_test_df['cleaned_text'] = processed_test_df.discourse_text.apply(clean_text)

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 45.4 s, sys: 2.51 s, total: 47.9 s
Wall time: 48.2 s


## TFIDF Vectorizer

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1000)
tfidf_vect.fit(new_train_df['cleaned_text'])

TfidfVectorizer(max_features=1000, token_pattern='\\w{1,}')

In [ ]:
train_tfidf = tfidf_vect.fit_transform(new_train_df['cleaned_text'])
test_tfidf = tfidf_vect.transform(processed_test_df['cleaned_text'])


In [ ]:
train_tfidf

<115691x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1623578 stored elements in Compressed Sparse Row format>

In [ ]:
#Upsampling the Train data
from imblearn.over_sampling import SMOTE

# Combine the TF-IDF data with the 'discourse_type' column
train_combined = pd.concat([pd.DataFrame(train_tfidf.toarray()), new_train_df['discourse_type']], axis=1)

# Apply SMOTE to upsample the data
smote = SMOTE(random_state=42)
X_upsampled, y_upsampled = smote.fit_resample(train_combined.drop(columns=['discourse_type']), train_combined['discourse_type'])

In [ ]:
features = tfidf_vect.get_feature_names_out()
features[:20]

array(['000', '1', '2', '2000', '2001', '25', '3', '4', '5', '6', '60',
       '7', 'a', 'ability', 'able', 'about', 'access', 'accident',
       'accidents', 'according'], dtype=object)

## Random Forest

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)

In [ ]:
%%time
rf_model.fit(X_upsampled, y_upsampled)

CPU times: user 47.6 s, sys: 1.56 s, total: 49.2 s
Wall time: 49.8 s


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [ ]:
rf_pred = rf_model.predict(test_tfidf)

In [ ]:
test_tfidf

<65311x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 490068 stored elements in Compressed Sparse Row format>

In [ ]:
test_preds = pd.DataFrame()
test_preds['id'] = processed_test_df['id']
test_preds['discourse_type'] = rf_pred
test_preds['predictionstring'] = processed_test_df['predictionstring']
test_preds['discourse_text']=processed_test_df['discourse_text']
test_preds.head()

,id,discourse_type,predictionstring,discourse_text
0,8CAF1B7BD82D,Claim,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18,As technology advances in a very constant and ...
1,8CAF1B7BD82D,Evidence,19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 3...,While we still might be waiting for the next f...
2,8CAF1B7BD82D,Evidence,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 6...,An example of this is a computer software that...
3,8CAF1B7BD82D,Rebuttal,62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 7...,Profesor Thomas Huang from the Beckman Institu...
4,8CAF1B7BD82D,Claim,97 98 99 100 101 102 103 104 105 106 107 108 1...,"Among the several other practical uses, Huang ..."


In [ ]:
id_list=list(test_preds['id'].unique())

In [ ]:
test_preds_temp = test_preds.copy()

In [ ]:
#Merging all test prediction strings based on id and discourse type
test_preds_temp['predictionstring_merged'] = test_preds.groupby(['id', 'discourse_type'])['predictionstring'].transform('sum')
test_merged = test_preds_temp.drop_duplicates(subset=['id','discourse_type','predictionstring_merged'])[['id','discourse_type','predictionstring_merged']]
test_merged.head(20)

,id,discourse_type,predictionstring_merged
0,8CAF1B7BD82D,Claim,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,8CAF1B7BD82D,Evidence,19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 3...
3,8CAF1B7BD82D,Rebuttal,62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 7...
5,8CAF1B7BD82D,Counterclaim,121 122 123 124 125 126 127 128 129 130 131 13...
6,8CAF1B7BD82D,Position,172 173 174 175 176 177 178 179 180 181 182 18...
12,8CAF1B7BD82D,Concluding Statement,390 391 392 393 394 395 396 397 398 399 400 40...
33,FD361491E6FA,Claim,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
42,FD361491E6FA,Evidence,275 276 277 278 279 280 281 282 283 284 285 28...
44,B71521CCA4F4,Claim,0 1 2 3 4 5 6 7 822 23 24 25 26 2728 29 30 31 ...
45,B71521CCA4F4,Lead,9 10 11 12 13 14 15 16 17 18 19 20 2146 47 48 ...


In [ ]:
train_df=train[train['id'].isin(id_list)]

In [ ]:
#Merging all train prediction strings based on id and discourse type
train_df['predictionstring_merged'] = train_df.groupby(['id', 'discourse_type'])['predictionstring'].transform('sum')
train_merged = train_df.drop_duplicates(subset=['id','discourse_type','predictionstring_merged'])[['id','discourse_type','predictionstring_merged']]
train_merged.head(20)

<ipython-input-32-b7cde4f01bfd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['predictionstring_merged'] = train_df.groupby(['id', 'discourse_type'])['predictionstring'].transform('sum')


,id,discourse_type,predictionstring_merged
92,4F0E197053FF,Position,0 1 2 3 4 5 6 7 8 9 10 11 12 13
93,4F0E197053FF,Lead,14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 2...
94,4F0E197053FF,Claim,106 107 108 109 110 111 112 113 114 115 116 11...
95,4F0E197053FF,Evidence,162 163 164 165 166 167 168 169 170 171 172 17...
96,4F0E197053FF,Counterclaim,259 260 261 262 263 264 265 266 267 268 269 27...
98,4F0E197053FF,Concluding Statement,395 396 397 398 399 400 401 402 403 404 405 40...
99,C3811E7F1750,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
100,C3811E7F1750,Position,80 81 82 83 84 85 86 87 88 89 90 91 92
101,C3811E7F1750,Claim,93 94 95 96 97 98 99 100 101 102 103 104 105 1...
102,C3811E7F1750,Evidence,114 115 116 117 118 119 120 121 122 123 124 12...


In [ ]:
final_df = pd.merge(train_merged, test_merged, how='left', on=['id','discourse_type'])
final_df.head()

,id,discourse_type,predictionstring_merged_x,predictionstring_merged_y
0,4F0E197053FF,Position,0 1 2 3 4 5 6 7 8 9 10 11 12 13,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,4F0E197053FF,Lead,14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 2...,NaN
2,4F0E197053FF,Claim,106 107 108 109 110 111 112 113 114 115 116 11...,41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 5...
3,4F0E197053FF,Evidence,162 163 164 165 166 167 168 169 170 171 172 17...,221 222 223 224 225 226 227 228 229 230 231 23...
4,4F0E197053FF,Counterclaim,259 260 261 262 263 264 265 266 267 268 269 27...,NaN


In [ ]:
#Final df with actuals and predicted values
final_df['predictionstring'] = final_df['predictionstring_merged_x']
final_df.drop(columns='predictionstring_merged_x', inplace=True)

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

def sim(str1, str2):
    return fuzz.partial_ratio(str1, str2)

def determine_cm(row):
    if pd.notna(row['predictionstring_merged_y']):
        similarity_score = sim(str(row['predictionstring']), str(row['predictionstring_merged_y']))
        if similarity_score > 0.5:
            return 'True Positive'
        else:
            return 'False Positive'
    else:
        return 'False Negative'

final_df['cm'] = final_df.apply(determine_cm, axis=1)

final_df.head(10)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,id,discourse_type,predictionstring_merged_y,predictionstring,cm
0,4F0E197053FF,Position,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0 1 2 3 4 5 6 7 8 9 10 11 12 13,True Positive
1,4F0E197053FF,Lead,NaN,14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 2...,False Negative
2,4F0E197053FF,Claim,41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 5...,106 107 108 109 110 111 112 113 114 115 116 11...,False Positive
3,4F0E197053FF,Evidence,221 222 223 224 225 226 227 228 229 230 231 23...,162 163 164 165 166 167 168 169 170 171 172 17...,False Positive
4,4F0E197053FF,Counterclaim,NaN,259 260 261 262 263 264 265 266 267 268 269 27...,False Negative
5,4F0E197053FF,Concluding Statement,NaN,395 396 397 398 399 400 401 402 403 404 405 40...,False Negative
6,C3811E7F1750,Lead,NaN,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,False Negative
7,C3811E7F1750,Position,94 95 96 97 98 99 100 101 102 103 104 105 106 107,80 81 82 83 84 85 86 87 88 89 90 91 92,True Positive
8,C3811E7F1750,Claim,0 1 23 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...,93 94 95 96 97 98 99 100 101 102 103 104 105 1...,False Positive
9,C3811E7F1750,Evidence,214 215 216 217 218 219 220 221 222 223 224 22...,114 115 116 117 118 119 120 121 122 123 124 12...,False Positive


In [ ]:
final_df_lead = final_df[final_df['discourse_type'] == 'Lead']
final_df.head(10)
final_df_claim = final_df[final_df['discourse_type'] == 'Claim']
final_df_Evidence = final_df[final_df['discourse_type'] == 'Evidence']
final_df_Position = final_df[final_df['discourse_type'] == 'Position']
final_df_Rebuttal = final_df[final_df['discourse_type'] == 'Rebuttal']
final_df_Concluding_Statement = final_df[final_df['discourse_type'] == 'Concluding Statement']
final_df_Counterclaim = final_df[final_df['discourse_type'] == 'Counterclaim']

In [ ]:
tp = final_df.loc[final_df['cm'] == 'True Positive'].shape[0]
fp = final_df.loc[final_df['cm'] == 'False Positive'].shape[0]
fn = final_df.loc[final_df['cm'] == 'False Negative'].shape[0]
f1_score = 2*tp/(2*tp + fp + fn)
print("F1-score:",f1_score)

F1-score: 0.6723746286912459


In [ ]:
# Calculate precision
precision = tp / (tp + fp)
# Calculate recall
recall = tp / (tp + fn)
# Print the scores
print("Precision:", precision)
print("Recall:", recall)

Precision: 0.6987470492100962
Recall: 0.6479205253409664


In [ ]:
def calculate_f1_score(df):
    tp = df.loc[df['cm'] == 'True Positive'].shape[0]
    fp = df.loc[df['cm'] == 'False Positive'].shape[0]
    fn = df.loc[df['cm'] == 'False Negative'].shape[0]
    f1_score = 2 * tp / (2 * tp + fp + fn)
    return f1_score

# Calculate F1 scores for each DataFrame
f1_score_lead = calculate_f1_score(final_df_lead)
f1_score_claim = calculate_f1_score(final_df_claim)
f1_score_evidence = calculate_f1_score(final_df_Evidence)
f1_score_position = calculate_f1_score(final_df_Position)
f1_score_rebuttal = calculate_f1_score(final_df_Rebuttal)
f1_score_concluding_statement = calculate_f1_score(final_df_Concluding_Statement)
f1_score_Counterclaim = calculate_f1_score(final_df_Counterclaim)
f1_score_overall = calculate_f1_score(final_df)
# Print or use the F1 scores as needed
print("F1 Score for Lead:", f1_score_lead)
print("F1 Score for Claim:", f1_score_claim)
print("F1 Score for Evidence:", f1_score_evidence)
print("F1 Score for Position:", f1_score_position)
print("F1 Score for Rebuttal:", f1_score_rebuttal)
print("F1 Score for Concluding_Statement:", f1_score_concluding_statement)
print("F1 Score for Counterclaim:", f1_score_Counterclaim)
print("F1 Score for Overall:", f1_score_overall)

F1 Score for Lead: 0.5449526813880127
F1 Score for Claim: 0.6972356296621325
F1 Score for Evidence: 0.4764720374476472
F1 Score for Position: 0.8929475587703436
F1 Score for Rebuttal: 0.7110694183864915
F1 Score for Concluding_Statement: 0.5516680227827502
F1 Score for Counterclaim: 0.8021978021978022
F1 Score for Overall: 0.6723746286912459


In [ ]:
id_list = list(test_preds['id'].unique())

In [ ]:
test_preds['discourse_start'] = test_preds['predictionstring'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) else None)
test_preds['discourse_end'] = test_preds['predictionstring'].apply(lambda x: int(x.split()[-1]) if isinstance(x, str) else None)

In [ ]:
import spacy
import random
from spacy import displacy

colors = {
    'Lead': '#8000ff',
    'Position': '#2b7ff6',
    'Evidence': '#2adddd',
    'Claim': '#80ffb4',
    'Concluding Statement': '#d4dd80',
    'Counterclaim': '#ff8042',
    'Rebuttal': '#ff0000'
}

nlp = spacy.load('en_core_web_sm')

def read_essay(id):
    with open(f"/content/drive/MyDrive/feedback-prize-2021/train/{id}.txt") as f:
        essay = f.read()
    return essay

def word_to_char_offsets(text):
    doc = nlp(text)
    char_offsets = [(token.idx, token.idx + len(token.text)) for token in doc]
    return char_offsets

def visualize(example):
    ents = []
    for i, row in test_preds[test_preds['id'] == example].iterrows():
        start_word_num = int(row['discourse_start']) if pd.notna(row['discourse_start']) else None
        end_word_num = int(row['discourse_end']) if pd.notna(row['discourse_end']) else None

        if start_word_num is not None and end_word_num is not None:
            # Get the character offsets for each word in the essay
            char_offsets = word_to_char_offsets(read_essay(example))

            # Get the start and end character positions of the entity
            start_char_pos = char_offsets[start_word_num][0]
            end_char_pos = char_offsets[end_word_num][1]

            if start_char_pos is not None and end_char_pos is not None:
                ents.append({
                    'start': start_char_pos,
                    'end': end_char_pos,
                    'label': row['discourse_type']
                })

    data = read_essay(example)

    doc2 = {
        "text": data,
        "ents": ents,
        "title": example
    }

    options = {"ents": test_preds.discourse_type.unique().tolist(), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

def visualize_random_example():
    random_id = random.choice(id_list)
    visualize(random_id)

# Call the function to visualize a random example
visualize_random_example()